In [2]:
##w word2vec을 쓰기 위해서 gensim을 다운받아야 한다
## terminal에 easy_install -U gensim 혹은 pip install --upgrade gensim을 친다

#설치오류시 참고
##https://blog.naver.com/sans223/221274010123
##https://blog.naver.com/ddonae_/221190968528

from glob import glob
from codecs import open as codecs_open
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
from re import sub
import re
from gensim.models.word2vec import Word2Vec
from os import path, mkdir
from multiprocessing import cpu_count
from sklearn.manifold import TSNE
from pandas import DataFrame
from matplotlib import pyplot as plt

C:\Users\Chankoo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
## 경로를 지정하고 파일을 읽어온다 바탕화면에 다운로드 할 경우 Users와 user를 본인 컴에 맞게 지정할 것

def read_books(location):
    if path.exists(location):
        return sorted(glob(path.join(location, "*.txt")))
    else:
        raise NotADirectoryError(location)
books = read_books(r'''books''')
read_books(r'''books''')

['books\\HPBook1.txt',
 'books\\HPBook2.txt',
 'books\\HPBook3.txt',
 'books\\HPBook4.txt',
 'books\\HPBook5.txt',
 'books\\HPBook6.txt',
 'books\\HPBook7.txt',
 'books\\HPBook8.txt']

In [4]:
## corpus 생성
def create_corpus(books):
    raw_corpus = u''
    for book in books:
        print("Reading {0}".format(book))
        with codecs_open(book, 'r', 'utf-8') as book_file:
            raw_corpus += book_file.read()
        print("Corpus is now {0} characters long".format(len(raw_corpus)))
    return raw_corpus
raw_corpus2 = create_corpus(books)

##뒤에 불용어 처리할때 (stop_word) 소문자 감안 하여 미리 소문자로 바꿔준다 (현중 : 나중에 바꾸는 것을 추천, 대문자 필요할 때 있으니까)
raw_corpus = raw_corpus2.lower()

Reading books\HPBook1.txt
Corpus is now 449988 characters long
Reading books\HPBook2.txt
Corpus is now 949340 characters long
Reading books\HPBook3.txt
Corpus is now 1575163 characters long
Reading books\HPBook4.txt
Corpus is now 2688627 characters long
Reading books\HPBook5.txt
Corpus is now 4204918 characters long
Reading books\HPBook6.txt
Corpus is now 5209621 characters long
Reading books\HPBook7.txt
Corpus is now 6404966 characters long
Reading books\HPBook8.txt
Corpus is now 6681889 characters long


In [5]:
## 문장 단위의 token을 생성 
def tokenize_corpus(raw_corpus):
    tokenizer = PunktSentenceTokenizer()
    return tokenizer.tokenize(raw_corpus)

tokenize_corpus(raw_corpus)
token = tokenize_corpus(raw_corpus)



In [6]:
#문장 단위의 token을 word로 쪼갠다
def sentence_to_words_list(token):
    stop_words = stopwords.words('english')
    words = re.sub("[^a-zA-Z]", " ", str(token)).split()
    return [word for word in words if word not in stop_words]
tokens = sentence_to_words_list(token)
sentence_to_words_list(token)[:10]

['chapter', 'one', 'r', 'n', 'r', 'n', 'r', 'nthe', 'boy', 'lived']

In [8]:
## 세글자 이상의 단어만 수집하기로 한다(r, n 이런 것 때문에!) 시간이 걸린다 
def tokens_to_words(tokens):
    words = [sentence_to_words_list(token) for token in tokens if len(token) > 2]
    print("The corpus contains {0:,} tokens".format(sum([len(word) for word in words])))
    return words

words = tokens_to_words(tokens)

The corpus contains 598,608 tokens


In [9]:
## workers = multiprocessing.cpu_count(), 
##min_count = 50 등등장횟수 50이하인 단어는 제외, 
##size=100 100개의 차원으로 embedding
## sg=0 이면 CBOW sg=1이면 skip.gram  (sg=skip.gram)

def we_build_vocab(words, num_features, min_word_count, num_workers, context_size):
    word2vec = Word2Vec(sg=1, 
                        size=num_features,   
                        min_count=min_word_count, 
                        workers=num_workers,
                        window=context_size)
    print("Building Vocabulary")
    word2vec.build_vocab(words)
    return word2vec

In [10]:
num_core = cpu_count() #cpu코어수
model = we_build_vocab(words,100,10,num_core,10)

Building Vocabulary


In [15]:
model.train(words,total_examples=model.corpus_count,epochs=10)

(5058333, 5986080)

In [16]:
##모델을 저장하고 불러와서 다시 training시킬 수 있다
model.save(r'''model_HP.w2v''')

In [17]:
model = Word2Vec.load(r'''model_HP.w2v''')

In [18]:
#'볼드모트' 를 임베딩한 벡터
model.wv['voldemort']

array([ 0.00137161,  0.00440795, -0.00256591, -0.00267156, -0.00301022,
        0.00051129, -0.00040498,  0.00045905, -0.00077698, -0.00403777,
        0.0039246 ,  0.00339091, -0.00205718,  0.00413822, -0.00055404,
       -0.00316601, -0.00080426, -0.00465249, -0.00059959, -0.00244771,
        0.00201681, -0.00018985,  0.00393345, -0.00410878, -0.00350473,
        0.00440481, -0.00124123,  0.00029857,  0.00168274,  0.00352092,
       -0.0011828 , -0.00208766, -0.00226902, -0.00141631,  0.0041537 ,
        0.00312662,  0.00434745, -0.00083323, -0.004753  ,  0.00363353,
        0.0001323 ,  0.00100896,  0.00196907,  0.00467548, -0.00274274,
       -0.00090765, -0.00368002,  0.00165301,  0.00070999, -0.0025635 ,
       -0.00074786, -0.00459201,  0.00388561,  0.00145363,  0.0030401 ,
        0.00348675, -0.00072612,  0.00465468,  0.00200209,  0.00018606,
       -0.00179823, -0.0014523 , -0.00334594,  0.00361283,  0.00280403,
       -0.00313029,  0.00335987,  0.00129711,  0.00495531, -0.00

In [19]:
## word_vector와 내장함수로 여러가지를 구해볼 수 있다
def word_correlation(word_vector, a, b, c):
    return word_vector.most_similar_cosmul(positive=[a, c], negative=[b])[0][0]


def word_find_most_similar(word_vector, word):
    return word_vector.most_similar(word)[0][0]


def word_odd_one(word_vector, phrase):
    return word_vector.doesnt_match(phrase.split())

def similarity(word_vector,a,b):
    return word_vector.similarity(a,b)



In [20]:
## 현재 학습이 제대로 안됨
print(word_correlation(model.wv, 'harry', 'voldemort', 'ron'))
print(word_find_most_similar(model.wv, 'ron'))
print(word_odd_one(model.wv,'He had been hugged by a complete stranger'))
print(similarity(model.wv,'harry','ron'))

gathering
north
stranger
-0.006215989860382375
